# Multi-class Classifier

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random

2025-06-30 17:42:42.303015: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751285562.395551   10108 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751285562.420107   10108 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751285562.620296   10108 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751285562.620341   10108 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751285562.620345   10108 computation_placer.cc:177] computation placer alr

In [2]:
BASE_DIR='../data/rps'
rock_dir = os.path.join(BASE_DIR, 'rock')
paper_dir = os.path.join(BASE_DIR, 'paper')
scissors_dir = os.path.join(BASE_DIR, 'scissors')

In [7]:
TRAINING_DIR = "../data/rps"
VALIDATION_DIR = "../data/rps-test-set"

train_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAINING_DIR,
    image_size=(150, 150),
    batch_size=32,
    label_mode='categorical'
    )
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    VALIDATION_DIR,
    image_size=(150, 150),
    batch_size=32,
    label_mode='categorical'
    )

SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE

train_dataset_final = (train_dataset
                       .cache()
                       .shuffle(SHUFFLE_BUFFER_SIZE)
                       .prefetch(PREFETCH_BUFFER_SIZE)
                       )

validation_dataset_final = (validation_dataset
                            .cache()
                            .prefetch(PREFETCH_BUFFER_SIZE)
                            )

Found 2520 files belonging to 3 classes.
Found 372 files belonging to 3 classes.


In [3]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(150, 150, 3)),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()

2025-06-30 20:31:00.317615: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-06-30 20:31:00.440027: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 169869312 exceeds 10% of free system memory.
2025-06-30 20:31:00.542225: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 169869312 exceeds 10% of free system memory.
2025-06-30 20:31:00.630131: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 169869312 exceeds 10% of free system memory.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 148, 148, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 82944)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 82944)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    42,467,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,508,099 (162.16 MB)

 Trainable params: 42,508,099 (162.16 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
data_augmentation = tf.keras.Sequential([
    tf.keras.Input(shape=(150, 150, 3)),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.4),
    tf.keras.layers.RandomTranslation(0.2,0.2),
    tf.keras.layers.RandomContrast(0.4),
    tf.keras.layers.RandomZoom(0.2)
    ])
model_with_aug = tf.keras.models.Sequential([
    data_augmentation,
    model
])

In [6]:
model_with_aug.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [8]:
history = model_with_aug.fit(
    train_dataset_final,
    epochs=5,
    validation_data = validation_dataset_final,
    verbose = 1
)

Epoch 1/5


2025-06-30 20:34:01.971807: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 169869312 exceeds 10% of free system memory.
2025-06-30 20:34:03.374255: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 169869312 exceeds 10% of free system memory.


79/79 ━━━━━━━━━━━━━━━━━━━━ 161s 2s/step - accuracy: 0.3302 - loss: 5.7293 - val_accuracy: 0.4973 - val_loss: 1.0629
Epoch 2/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 0.3858 - loss: 1.2169 - val_accuracy: 0.5403 - val_loss: 1.0116
Epoch 3/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.4399 - loss: 1.0546 - val_accuracy: 0.6344 - val_loss: 0.8578
Epoch 4/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 150s 2s/step - accuracy: 0.5218 - loss: 0.9729 - val_accuracy: 0.7070 - val_loss: 0.6486
Epoch 5/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.5591 - loss: 0.8839 - val_accuracy: 0.7419 - val_loss: 0.5427
